In [1]:
import os

import cv2
import geopandas as gpd
import pandas as pd
import numpy as np
np.random.seed(42)
import rasterio
import shapely.geometry
from scipy.misc import imresize
import pickle



In [2]:
# Load the image summary
image_summary = gpd.read_file('vectors/shanghai_image_summary.geojson')

# Load the landuse data from OSM
osm_landuse = gpd.read_file('vectors/shanghai_landuse.geojson')
osm_leisure = gpd.read_file('vectors/shanghai_leisure_RGB.geojson')

# Convert everything to polygons
osm_landuse.set_geometry(osm_landuse.geometry.apply(shapely.geometry.Polygon), inplace=True)
osm_leisure.set_geometry(osm_leisure.geometry.apply(shapely.geometry.Polygon), inplace=True)

# Limit to just farmland
farmland = osm_landuse[osm_landuse.landuse == 'farmland'].unary_union

files = [gpd.read_file(os.path.join('vectors/new-geojson', x)) for x in os.listdir('vectors/new-geojson')]
manual_farmland = gpd.GeoDataFrame( pd.concat( files, ignore_index=True) )
manual_farmland.set_geometry(manual_farmland.geometry.apply(shapely.geometry.Polygon), inplace=True)
all_manual_farmland = manual_farmland.unary_union

with open('merged_farmland.pkl', 'rb') as f:
    merged_farmland = pickle.load(f)
with open('all_manual_farmland.pkl', 'rb') as f:
    all_manual_farmland = pickle.load(f)
with open('all_farmland.pkl', 'rb') as f:
    all_farmland = pickle.load(f)

# Other categories (add in vectors)
farmyard = osm_landuse[osm_landuse.landuse == 'farmyard'].unary_union
industrial = osm_landuse[osm_landuse.landuse == 'industrial'].unary_union
park = osm_leisure[osm_leisure.leisure == 'park'].unary_union

vectors = [merged_farmland]

In [3]:
INPUT_SIZE = 224

def polycoords(poly):
    """Convert a polygon into the format expected by OpenCV
    """
    if poly.type in ['MultiPolygon', 'GeometryCollection']:
        return [np.array(p.exterior.coords) for p in poly if p.type == 'Polygon']
    elif poly.type == 'Polygon':
        return [np.array(poly.exterior.coords)]
    else:
        print('Encountered unrecognized geometry type {}. Ignoring.'.format(poly.type))
        return []
    
def make_mask(img_shape, poly):
    """Make a mask from a polygon"""
    poly_pts = polycoords(poly)
    polys = [x.astype(int) for x in poly_pts]
    # Create an empty mask and then fill in the polygons
    mask = np.zeros(img_shape[:2])
    cv2.fillPoly(mask, polys, 255)
    return mask.astype('uint8')

def scale_bands(img, lower_pct=1, upper_pct=99):
    """Rescale the bands of a multichannel image for display"""
    img_scaled = np.zeros(img.shape, np.uint8)
    for i in range(img.shape[2]):
        band = img[:, :, i]
        lower, upper = np.percentile(band, [lower_pct, upper_pct])
        band = (band - lower) / (upper - lower) * 255
        img_scaled[:, :, i] = np.clip(band, 0, 255).astype(np.uint8)
    return img_scaled

def resize(img, new_shape):
    img_resized = np.zeros(new_shape+(img.shape[2],)).astype('float32')
    for i in range(img.shape[2]):
        img_resized[:, :, i] = imresize(img[:, :, i], new_shape, interp='bicubic')
    return img_resized

# Build a training set
def make_set(image_summary, vectors, training_set_size, input_size, rows_to_use, channels=8):
    
    X = []
    X_val = []
    Y = []
    Y_val = []
    
    for i, row in image_summary.loc[rows_to_use].iterrows():
        with rasterio.open(row.image_name) as src:
            if channels == 'bgr':
                img = src.read([2, 3, 5]).transpose([1, 2, 0]) # BGR for VGG
            elif channels == 'rgb':
                img = src.read([5, 3, 2]).transpose([1, 2, 0]) 
            else:
                img = src.read().transpose([1, 2, 0])
            img_bounds = shapely.geometry.box(*src.bounds)
            img_transform = list(np.array(~src.transform)[[0, 1, 3, 4, 2, 5]])
        
        # Ignore faulty images
        if np.sum(img[:,:,2]==0) < 500:
           
            masks = []


            for i, poly in enumerate(vectors):
                
                # Get the intersection between the polygon and the image bounds

                if all_manual_farmland.intersection(img_bounds):
                    mask_poly = all_manual_farmland.intersection(img_bounds)
                else:
                    mask_poly = all_farmland.intersection(img_bounds)

                # Transform it into pixel coordinates
                mask_poly_pxcoords = shapely.affinity.affine_transform(mask_poly, img_transform)

                # Convert the polygon into a mask
                mask = make_mask(img.shape[:2], mask_poly_pxcoords)
                mask = imresize(mask, (INPUT_SIZE, INPUT_SIZE))

                masks.append(mask[..., None])
            masks = np.concatenate(masks, axis=2)
            img = resize(img, (input_size, input_size))

            # Add each mask to a list
            X.append(img[None, ...])
            Y.append(masks[None, ...])
    
    # Build validation set
    for i, row in image_summary.iterrows():
        if i not in rows_to_use:
            with rasterio.open(row.image_name) as src:
                if channels == 'bgr':
                    img = src.read([2, 3, 5]).transpose([1, 2, 0]) # BGR for VGG
                elif channels == 'rgb':
                    img = src.read([5, 3, 2]).transpose([1, 2, 0]) 
                else:
                    img = src.read().transpose([1, 2, 0])
                img_bounds = shapely.geometry.box(*src.bounds)
                img_transform = list(np.array(~src.transform)[[0, 1, 3, 4, 2, 5]])

            # Ignore faulty images
            if np.sum(img[:,:,2]==0) < 500:

                masks = []

                for i, poly in enumerate(vectors):
                    
                    # Get the intersection between the polygon and the image bounds
                    if all_manual_farmland.intersection(img_bounds):
                        mask_poly = all_manual_farmland.intersection(img_bounds)
                    else:
                        mask_poly = all_farmland.intersection(img_bounds)

                    # Transform it into pixel coordinates
                    mask_poly_pxcoords = shapely.affinity.affine_transform(mask_poly, img_transform)

                    # Convert the polygon into a mask
                    mask = make_mask(img.shape[:2], mask_poly_pxcoords)
                    mask = imresize(mask, (INPUT_SIZE, INPUT_SIZE))

                    masks.append(mask[..., None])
                masks = np.concatenate(masks, axis=2)
                img = resize(img, (input_size, input_size))

                # Add each mask to a list
                X_val.append(img[None, ...])
                Y_val.append(masks[None, ...])
            
    # Concatenate the results
    X = np.concatenate(X, axis=0)
    Y = np.concatenate(Y, axis=0)

    X_val = np.concatenate(X_val, axis=0)
    Y_val = np.concatenate(Y_val, axis=0)
    
    # Normalize the values
    X = X.astype('float32')
    X = (X / X.max() - 0.5) * 2 # put X in range [-1, 1]

    Y = Y.astype('float32') / 255 # put Y in range [0, 1]
        
    X_val = X_val.astype('float32')
    X_val = (X_val / X_val.max() - 0.5) * 2 # put X in range [-1, 1]
    
    Y_val = Y_val.astype('float32') / 255 # put Y in range [0, 1]
    
    return X, Y, X_val, Y_val    

In [4]:
rows_to_use = np.random.choice(image_summary.index, 4000, replace=False)

In [5]:
X_train_vgg, Y_train_vgg, X_val_vgg, Y_val_vgg = make_set(image_summary, vectors, 4000, INPUT_SIZE, rows_to_use, channels='bgr')

In [6]:
X_train_unet, Y_train_unet, X_val_unet, Y_val_unet = make_set(image_summary, vectors, 4000, INPUT_SIZE, rows_to_use, channels=8)

In [7]:
X_train_resnet, Y_train_resnet, X_val_resnet, Y_val_resnet = make_set(image_summary, vectors, 4000, INPUT_SIZE, rows_to_use, channels='rgb')

In [8]:
X_train_unet_256, Y_train_unet_256, X_val_unet_256, Y_val_unet_256 = make_set(image_summary, vectors, 4000, INPUT_SIZE, rows_to_use, channels=8)

In [56]:
X_train_vgg.shape

(3622, 224, 224, 3)

In [57]:
X_val_vgg.shape

(517, 224, 224, 3)

In [58]:
X_train_unet.shape

(3622, 224, 224, 8)

In [59]:
X_val_unet.shape

(517, 224, 224, 8)

In [60]:
Y_train_vgg.shape

(3622, 224, 224, 1)

In [61]:
Y_train_unet.shape

(3622, 224, 224, 1)

In [62]:
Y_val_vgg.shape

(517, 224, 224, 1)

In [63]:
Y_val_unet.shape

(517, 224, 224, 1)

In [22]:
import pickle

In [64]:
with open('mn_xtrain_224_bgr_man.pkl', 'wb') as f:
    pickle.dump(X_train_vgg, f, pickle.HIGHEST_PROTOCOL)

In [65]:
with open('mn_xtrain_224_8channel_man.pkl', 'wb') as f:
    pickle.dump(X_train_unet, f, pickle.HIGHEST_PROTOCOL)

In [66]:
with open('mn_ytrain_224_man.pkl', 'wb') as f:
    pickle.dump(Y_train_unet, f, pickle.HIGHEST_PROTOCOL)

In [67]:
with open('mn_xval_224_bgr_man.pkl', 'wb') as f:
    pickle.dump(X_val_vgg, f, pickle.HIGHEST_PROTOCOL)

In [68]:
with open('mn_xval_224_8channel_man.pkl', 'wb') as f:
    pickle.dump(X_val_unet, f, pickle.HIGHEST_PROTOCOL)

In [69]:
with open('mn_yval_224_man.pkl', 'wb') as f:
    pickle.dump(Y_val_unet, f, pickle.HIGHEST_PROTOCOL)

In [10]:
with open('mn_xtrain_256_rgb_man.pkl', 'wb') as f:
    pickle.dump(X_train_resnet, f, pickle.HIGHEST_PROTOCOL)

In [11]:
with open('mn_ytrain_256_man.pkl', 'wb') as f:
    pickle.dump(Y_train_resnet, f, pickle.HIGHEST_PROTOCOL)

In [12]:
with open('mn_xval_256_rgb_man.pkl', 'wb') as f:
    pickle.dump(X_val_resnet, f, pickle.HIGHEST_PROTOCOL)

In [13]:
with open('mn_yval_256_man.pkl', 'wb') as f:
    pickle.dump(Y_val_resnet, f, pickle.HIGHEST_PROTOCOL)

In [7]:
with open('mn_xtrain_256_8channel_man.pkl', 'wb') as f:
    pickle.dump(X_train_unet_256, f, pickle.HIGHEST_PROTOCOL)

In [8]:
with open('mn_xval_256_8channel_man.pkl', 'wb') as f:
    pickle.dump(X_val_unet_256, f, pickle.HIGHEST_PROTOCOL)